In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import codecs
import json
import os

data_dir = '/home/jvdzwaan/data/ocr/'
with codecs.open(os.path.join(data_dir, 'BAObi1.json'), encoding='utf-8') as f:
    result = json.load(f)
aligned = result['gs']

with codecs.open(os.path.join(data_dir, 'BAObi1-ds.gs.txt'), encoding='utf-8') as f:
    text = f.read()
print text

with codecs.open(os.path.join(data_dir, 'BAObi1-ds.ocr.txt'), encoding='utf-8') as f:
    ocr = f.read()
print ocr

In [ ]:
from pattern.nl import parsetree

def parse(text):
    tokens = []
    p = parsetree(text,
                  tokenize=True,     # Split punctuation marks from words?
                  tags=True,         # Parse part-of-speech tags? (NN, JJ, ...)
                  chunks=False,      # Parse chunks? (NP, VP, PNP, ...)
                  relations=False,   # Parse chunk relations? (-SBJ, -OBJ, ...)
                  lemmata=True,      # Parse lemmata? (ate => eat)
                  encoding='utf-8',  # Input string encoding.
                  tagset=None)       # Penn Treebank II (default) or UNIVERSAL.
    for sentence_id, sentence in enumerate(p):
        for word_id, word in enumerate(sentence):
            tokens.append({'id': word_id,
                           'word': word.string,
                           'lemma': word.lemma,
                           'sentence': sentence_id,
                           'pos': word.type})
    return tokens

tokens = [t['word'] for t in parse(text)]

In [ ]:
print tokens

In [ ]:
unaligned = []

for w in tokens:
    for c in w:
        unaligned.append(c)
    unaligned.append('@')

i = 0
j = 0
wb = []
while True:
    if unaligned[j] == '@':
        print 'word boundary', i
        wb.append(i)
        j += 1
    elif aligned[i] == '' or aligned[i] == ' ':
        i += 1
    elif aligned[i] == unaligned[j]:
        i += 1
        j += 1

In [ ]:
print wb
wb2 = []
p = 0
for b in wb:
    print '"{}"'.format(''.join(aligned[p:b]))
    w = ''.join(aligned[p:b])
    if w.startswith(' '):
        s = p + 1
    else:
        s = p
    e = b
    wb2.append((s, e))
    p = b
print wb2

In [ ]:
ocr_words = []
for b, e in wb2:
    #print b, e
    #print ''.join(ocr[b: e])
    ocr_words.append(''.join(ocr[b: e]))
print ocr_words
print len(ocr_words)

In [ ]:
import pandas as pd

df = pd.read_csv('/home/jvdzwaan/code/ocr/merged.csv', encoding='utf-8')
df

In [ ]:
import re
from string import punctuation

print punctuation
text = '..****&&&&$$$$!!!!..'

for c in punctuation:
    print c
    try:
        if c == u'.':
            text = re.sub(r'\{}+'.format(c), c, text)
        else:
            text = re.sub(r'{}+'.format(c), c, text)
    except:
        print 'exception'
        if c != u'\\':
            text = re.sub(r'\{}+'.format(c), c, text)
    print text
    